<a href="https://colab.research.google.com/github/Rafat97/my-tensorflow/blob/image/_tensorflow_image_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip list


# Image Load using keras.preprocessing





In [0]:
import tensorflow as tf
import matplotlib.pyplot as plt
import datetime
import IPython.display as display
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os
import pathlib


DATA_PATH = os.getcwd()+'/drive/My Drive/Colab Notebooks/ball-dataset';
data_dir = pathlib.Path(DATA_PATH)
print("dir = ",data_dir)

## number of image 
image_count = len(list(data_dir.glob('*/*.jpg')))
print("image_count = ",image_count)

CLASS_NAMES = np.array(['ball'])
print("CLASS_NAMES = ",CLASS_NAMES)

balls = list(data_dir.glob('ball/*'))
print(balls)

# for image_path in balls[:]:
#     display.display(Image.open(str(image_path)))

image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

BATCH_SIZE = 4
IMG_HEIGHT = 200
IMG_WIDTH = 200
STEPS_PER_EPOCH = np.ceil(image_count/BATCH_SIZE)

train_data_gen = image_generator.flow_from_directory(directory=str(data_dir),
                                                     batch_size=BATCH_SIZE,
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                     classes = list(CLASS_NAMES))

train_data_gen

def show_batch(image_batch, label_batch):
  plt.figure(figsize=(10,10))
  for n in range(4):
      ax = plt.subplot(5,5,n+1)
      plt.imshow(image_batch[n])
      plt.title(CLASS_NAMES[label_batch[n]==1][0].title())
      # plt.axis('off')

image_batch, label_batch = next(train_data_gen)

show_batch(image_batch, label_batch)




# Image Load using tf.data


In [0]:
import tensorflow as tf
import matplotlib.pyplot as plt
import datetime
import IPython.display as display
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os
import pathlib

AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 4
IMG_HEIGHT = 200
IMG_WIDTH = 200
STEPS_PER_EPOCH = 5
# np.ceil(image_count/BATCH_SIZE)

DATA_PATH = os.getcwd()+'/drive/My Drive/Colab Notebooks/ball-dataset';
data_dir = pathlib.Path(DATA_PATH)
print("dir = ",data_dir)

CLASS_NAMES = np.array(['ball'])
print("CLASS_NAMES = ",CLASS_NAMES)

list_ds = tf.data.Dataset.list_files(str(data_dir/'*/*.jpg'))
for f in list_ds.take(5):
  print(f.numpy())

def get_label(file_path):
  # convert the path to a list of path components
  parts = tf.strings.split(file_path, os.path.sep)
  # The second to last is the class-directo
  return parts[-2] == CLASS_NAMES


def decode_img(img):
  # convert the compressed string to a 3D uint8 tensor
  img = tf.image.decode_jpeg(img, channels=3)
  # Use `convert_image_dtype` to convert to floats in the [0,1] range.
  img = tf.image.convert_image_dtype(img, tf.float32)
  # resize the image to the desired size.
  return tf.image.resize(img, [IMG_WIDTH, IMG_HEIGHT])


def process_path(file_path):
  label = get_label(file_path)
  # load the raw data from the file as a string
  img = tf.io.read_file(file_path)
  img = decode_img(img)
  return img, label

def show_batch(image_batch, label_batch):
  plt.figure(figsize=(10,10))
  for n in range(4):
      ax = plt.subplot(5,5,n+1)
      plt.imshow(image_batch[n])
      plt.title(CLASS_NAMES[label_batch[n]==1][0].title())


def prepare_for_training(ds, cache=True, shuffle_buffer_size=1000):
  # This is a small dataset, only load it once, and keep it in memory.
  # use `.cache(filename)` to cache preprocessing work for datasets that don't
  # fit in memory.
  if cache:
    if isinstance(cache, str):
      ds = ds.cache(cache)
    else:
      ds = ds.cache()

  ds = ds.shuffle(buffer_size=shuffle_buffer_size)

  # Repeat forever
  ds = ds.repeat()

  ds = ds.batch(BATCH_SIZE)

  # `prefetch` lets the dataset fetch batches in the background while the model
  # is training.
  ds = ds.prefetch(buffer_size=AUTOTUNE)

  return ds


labeled_ds = list_ds.map(process_path, num_parallel_calls=AUTOTUNE)
# print(labeled_ds)

# for image, label in labeled_ds.take(1):
#   print("Image shape: ", image.numpy().shape)
#   print("Label: ", label.numpy())

train_ds = prepare_for_training(labeled_ds)
image_batch, label_batch = next(iter(train_ds))
show_batch(image_batch.numpy(), label_batch.numpy())

# Image classification

cats and dogs image classification 

In [0]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
import pathlib


In [0]:
# load data from url and get the stored path
_URL = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'
path_to_zip = tf.keras.utils.get_file('cats_and_dogs.zip', origin=_URL, extract=True)
PATH = os.path.join(os.path.dirname(path_to_zip), 'cats_and_dogs_filtered')
PATH

In [0]:
# separate train dir and validation dir
import pprint
data_dir = pathlib.Path(PATH)
dir_info = list(data_dir.glob('*'))
train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'validation')
print("data_dir = ",data_dir)
print("dir_info :")
pprint.pprint(dir_info)
print("train_dir = ",train_dir)
print("validation_dir = ",validation_dir)

In [0]:
# separate cats & dog dir
train_cats_dir = os.path.join(train_dir, 'cats')  
train_dogs_dir = os.path.join(train_dir, 'dogs')  
validation_cats_dir = os.path.join(validation_dir, 'cats')  
validation_dogs_dir = os.path.join(validation_dir, 'dogs')  

# separate cats & dog dir count
num_cats_tr = len(os.listdir(train_cats_dir))
num_dogs_tr = len(os.listdir(train_dogs_dir))
num_cats_val = len(os.listdir(validation_cats_dir))
num_dogs_val = len(os.listdir(validation_dogs_dir))
total_train = num_cats_tr + num_dogs_tr
total_val = num_cats_val + num_dogs_val

print("train_cats_dir = ",train_cats_dir)
print("train_dogs_dir = ",train_dogs_dir)
print("validation_cats_dir = ",validation_cats_dir)
print("validation_dogs_dir = ",validation_dogs_dir)
print("num_cats_tr = ",num_cats_tr)
print("num_dogs_tr = ",num_dogs_tr)
print("num_cats_val = ",num_cats_val)
print("num_dogs_val = ",num_dogs_val)
print("total_train = ",total_train)
print("total_val = ",total_val)

In [0]:
# Define some variable
batch_size = 128
epochs = 15
IMG_HEIGHT = 150
IMG_WIDTH = 150

In [0]:
# Generator for our training & validation data
train_image_generator = ImageDataGenerator(rescale=1./255)
validation_image_generator = ImageDataGenerator(rescale=1./255) 

In [0]:
# Generator for train data gen & validation data gen data
train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=train_dir,
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           class_mode='binary')

val_data_gen = validation_image_generator.flow_from_directory(batch_size=batch_size,
                                                              directory=validation_dir,
                                                              target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                              class_mode='binary')

In [0]:
# Visualize training images
sample_training_images, _ = next(train_data_gen)

In [0]:
# This function will plot images in the form of a grid with 1 row and 5 columns where images are placed in each column.
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 5, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
        # ax.axis('off')
    plt.tight_layout()
    plt.show()

plotImages(sample_training_images[:5])

In [0]:
#model create Conv2D->MaxPooling2D->Conv2D->MaxPooling2D->Conv2D->MaxPooling2D->Flatten->Dense->Dense
model = Sequential([
    Conv2D(16, 3, padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH ,3)),
    MaxPooling2D(),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1)
])

In [0]:
# Compile the mode with summary
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.summary()

In [0]:
#Train the model
history = model.fit_generator(
    train_data_gen,
    steps_per_epoch=total_train // batch_size,
    epochs=epochs,
    validation_data=val_data_gen,
    validation_steps=total_val // batch_size
)

In [0]:
history.history

In [0]:
# Visualize training results
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss=history.history['loss']
val_loss=history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()